In [ ]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display
import pandas as pd

from islets.Recording import Recording, saveMovie
from islets.Regions import Regions
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

from caiman import movie as cmovie

from matplotlib.colors import LogNorm

In [ ]:
# from copy import deepcopy
fracSaturTh = .05
# movieFreq = 2
cmap = plt.cm.Greys
cmap.set_bad("lime")
pd.set_option('display.max_rows', 100)

In [ ]:
plt.rcParams["animation.embed_limit"] = 500

In [ ]:
recFile = "/data/Sandra/2020/2020_07_08/Experiment59c.lif"

In [ ]:
rec = Recording(recFile)
rec.metadata

In [ ]:
ser = "Series003"#rec.metadata.Name[7]
# ser = "all"

In [ ]:
rec.import_series(ser)

In [ ]:
metadata = rec.Series[ser]['metadata']

In [ ]:
movie = cmovie(rec.Series[ser]['data'],fr = metadata.Frequency)

In [ ]:
movie.shape

In [ ]:
movie.fr

In [ ]:
5/metadata.pxSize

### Hand-curated ROIs

#### massage to get

In [ ]:
nRebin = 10
rebMovie = cmovie(rebin(movie[len(movie)//5:],nRebin), fr=movie.fr/nRebin)

In [ ]:
# pc_low  = np.percentile(np.abs(np.diff(rebMovie)),1,axis=0)
# pc_mid  = np.percentile(np.abs(np.diff(rebMovie)),50,axis=0)
# pc_high = np.percentile(np.abs(np.diff(rebMovie)),99,axis=0)

In [ ]:
10/metadata.pxSize

In [ ]:
from islets.utils import multi_map

In [ ]:
from copy import deepcopy

In [ ]:
from cv2 import GaussianBlur

In [ ]:
blurSize = 3
if blurSize>1:
    def iterf(xi):
        return GaussianBlur(xi.T,(blurSize,blurSize),-1).T
    nn = 100
    blurmovie = np.concatenate(multi_map(iterf, np.split(rebMovie[:len(rebMovie)//nn*nn],nn), processes=10))
else:
    blurmovie = rebMovie

dmovie = np.diff(blurmovie)

In [ ]:
pc_low  = np.percentile(np.abs(dmovie),1,axis=0)
pc_mid  = np.percentile(np.abs(dmovie),50,axis=0)
pc_high = np.percentile(np.abs(dmovie),99,axis=0)

In [ ]:
ims = [pc_mid, pc_high, pc_mid+pc_high]

In [ ]:
fig,axs = plt.subplots(2,3,
                       figsize=(16,8), 
                       gridspec_kw={'height_ratios': [3, 1]},
                       sharex="row",
                       sharey="row",
                      )
for ax, im, axl in zip(axs[0],ims,axs[1]):
    ax.imshow(im,
              vmin=min(map(np.min,ims))+1e-1,
              vmax=max(map(np.max,ims)),
              norm=LogNorm()
             )
    axl.hist(np.log10(1+im.flatten()),100,log=True)
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,6))
axs[0].imshow(pc_mid,
#            norm=LogNorm()
          )
axs[1].imshow(pc_mid,
           norm=LogNorm()
          )

In [ ]:
# rs = []
# for im in [pc_low, pc_high, pc_high+pc_low]:
#     rs += [Regions(im,diag=True,gSig_filt=None)]

In [ ]:
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph, plotRoi_to_be_connected

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
# image = pc_low+pc_high
image = pc_mid
# image = dmovie.std(0)
# image = GaussianBlur(image,(3,3),-1)

In [ ]:
im_th = get_sep_th(image.flatten(),log=True,plot=True)

In [ ]:
regions = Regions(image, gSig_filt=None, diag=True, img_th=im_th/image.max(), processes=1)

In [ ]:
peakTh = get_sep_th(regions.df.peakValue,plot=True,log=True)

In [ ]:
regions.df.drop(index=regions.df.query(f"peakValue<{peakTh}").index,inplace=True)
regions.update()

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.subplot(111)
regions.plotEdges(ax=ax)
regions.plotPeaks(ax=ax)

In [ ]:
while True:
    maxSize = np.percentile(regions.df["size"],80)
    sizeTh = maxSize/2
    peak2bnd = getPeak2BoundaryDF(regions.df)
    df = peak2bnd.query("dist<1.")[["i","j"]]
    if len(df)==0: break
    df=df[df.i.isin(regions.df.loc[df.i].query(f"size<{sizeTh}").index)]
    df=df[df.j.isin(regions.df.loc[df.j].query(f"size<{maxSize}").index)]
    if len(df)==0: break
    plt.figure(figsize=(10,10))
    ax = plt.subplot(111)
    regions.plotEdges(ax=ax,imkw_args={"cmap":"Greys","vmax":3}, color="blue")
    regions.plotPeaks(ax=ax, color="darkred")
    gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=True, ax=ax)
    dropped = mergeBasedOnGraph(gRois,regions)
    plt.show()
#     if dropped == 0: break

In [ ]:
# saveRois(
#     regions, 
#     f"{recFile}_analysis/{ser}", 
#     filename="srdjan_handcurated",
#     col=[],
#     formats=["vienna"]
#         )

In [ ]:
# for sizeTh in range(2,6):
#     if sum(regions.df['size']<sizeTh)<1: continue
#     th = get_sep_th(regions.df.query(f"size<{sizeTh}").interest,log=True, plot=True)
#     drop = regions.df.query(f"size<{sizeTh} and interest<{th}").index
#     regions.df.drop(index=drop, inplace=True)
#     regions.update()
#     peak2bnd = getPeak2BoundaryDF(regions.df)
#     df = peak2bnd.query("dist<1.")[["i","j"]]
#     # if len(df)==0: break
#     df=df[df.i.isin(regions.df.loc[df.i].query(f"size<{sizeTh}").index)]
#     gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=True)
#     dropped = mergeBasedOnGraph(gRois,regions)
#     # if dropped == 0: break
#     plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ax = plt.subplot(111)
regions.plotEdges(color="r",lw=.4,ax=ax)
regions.plotPeaks(color="r",ax=ax)
# regions.plotEdges(color="c",lw=.9,ix=regions.df.query("Nneighbors==0").index,ax=ax)
# regions.plotPeaks(color="c",ms=5,ix=regions.df.query("Nneighbors==0").index,ax=ax)

#### import saved

In [ ]:
ser = "Series002-5"

In [ ]:
pickleFilename = f"{recFile}_analysis/{ser}/"+[f for f in os.listdir(f"{recFile}_analysis/{ser}") if "handcur" in f][0]
with open(pickleFilename, "rb") as f:
    regions = pickle.load(f)

In [ ]:
# n0 = len(movie)//5
# regions.update(movie[n0:])

In [ ]:
regions.update(movie)

In [ ]:
regions.time = np.arange(len(movie))/regions.Freq

In [ ]:
regions.time += len(movie)/regions.Freq 

In [ ]:
# regions.time += n0/regions.Freq 

In [ ]:
regions.import_protocol(f"{recFile}_analysis/{ser}/"+[f for f in os.listdir(f"{recFile}_analysis/{ser}/") if "protocol" in f][0])

In [ ]:
regions.df.drop(index = regions.df[regions.df["size"]<30].index, inplace=True)

In [ ]:
for ts in [1,3,10,30,100,300]:
    regions.fast_filter_traces(ts,)

from scipy.stats import rankdata

regions.df["zInterest"] = np.max([rankdata((np.vstack(regions.df[c])>3.5).mean(1)) \
                                  for c in regions.df.columns if "zScore" in c],axis=0)

#### Fittting

In [ ]:
# regions.detrend_traces()

In [ ]:
sizes = regions.df['size'].values.copy()
sizes.sort()
plt.figure()
plt.plot(sizes,np.arange(len(sizes)))

In [ ]:
regions.detrend_traces(fast=False,timescale=100)

In [ ]:
regions.slow_filter_traces(100, percentile=[5],avg=False)

In [ ]:
regions.fast_filter_traces(3,saveIntermed=True,)

In [ ]:
regions.examine2(
    startShow=regions.df.sort_values("zInterest", ascending=False).index[:300]
#     startShow=regions.df.query("size>30").index
                )

In [ ]:
from islets.fitting import multiShape

In [ ]:
# %matplotlib inline
%matplotlib notebook

In [ ]:
from islets.fitting import fit_spikes

In [ ]:
for c in regions.df.columns:
    if "_" in c:
        del regions.df[c]

try: del regions.showTime
except: pass
try: del regions.peaks
except: pass

In [ ]:
regions.df.columns

In [ ]:
# ts = 3
# regions.fast_filter_traces(ts, z_sp=3, saveIntermed=True)

In [ ]:
regions.infer_gain(True)
regions.gain

In [ ]:
regions.gain = 1

In [ ]:
from copy import deepcopy

In [ ]:
regions.df["_trace"] = deepcopy(regions.df.trace)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:

t = regions.time.copy()
ts = 3
sl,fs,zs = regions.fast_filter_traces(ts, write=False, normalize=True, z_sp=3,verbose=True,Npoints=20)
try:
    tt = regions.showTime["%g"%ts]
except:
    tt = t.copy()
spikes = regions.calc_peaks(
    ts=ts,
    save=False,
    t=tt,
    zScores=zs,
    smooth=0
)
fitDF = []
fig, axs = plt.subplots(len(regions.df),1,figsize=(9,9), sharey=False, sharex=True)
for i,ix in enumerate(regions.df.index[:]):
    tmpdf = fit_spikes(
#         t, regions.df.trace[ix],
       tt, sl[i]+fs[i],
       spikes.query(f"roi=={ix}"),
#        ifit = 3,
       colorCode={"myShape":"C3"},
       plot=True,
       ax=axs[i],
       half_width_threshold=ts*.12
      )
    tmpdf["roi"]= ix 
    fitDF += [tmpdf]
#     axs[i].plot(tt,sl[i],"C1",lw=.8)
fitDF = pd.concat(fitDF)
# plt.xlim(regions.time[0], regions.time[0]+50)
plt.xlim(fitDF["loc"].min()-10, fitDF["loc"].min()+50)
plt.subplots_adjust(hspace=0.01)

In [ ]:
# fig = plt.figure(figsize=(10,3))
# fig.add_axes([0.05,0.1,.9,.9])
regions.df["_trace"] = [tr.copy() for tr in regions.df.trace]
# plt.plot(t,y,lw=.4)
for _,row in fitDF.iterrows():
    ix = row.roi
# for _,row in tmpdf.iterrows():
    y  = regions.df.loc[ix,"_trace"]
    func = eval(row.fshape)
    yf,bkg = func(t, *row.pars, bkgsum=False)
#     plt.plot(t,yf+bkg,"C1",lw=.4)
    ff = np.isfinite(yf)
#     plt.plot(t,bkg,"C1")
    y[ff] = bkg[ff]
    
# plt.xlim(tmpdf['loc'].min()-10, tmpdf['loc'].max()+20)

In [ ]:
for i,ix in enumerate(regions.df.index):
    axs[i].plot(t, regions.df.loc[ix,"_trace"], "C2",lw=.8)

In [ ]:
regions.df["_trace"] = [tr.copy() for tr in regions.df.trace]
t = regions.time.copy()
ts = 3

In [ ]:

sl,fs,zs = regions.fast_filter_traces(ts, write=False, normalize=True, z_sp=3,verbose=True,Npoints=20,usecol="_trace")
try:
    tt = regions.showTime["%g"%ts]
except:
    tt = t.copy()
spikes = regions.calc_peaks(
    ts=ts,
    save=False,
    t=tt,
    zScores=zs,
    smooth=0
)
print(len(spikes))
    
    
for _ in range(4):
    print (_)
    if len(spikes)==0: break
    sl,fs,zs = regions.fast_filter_traces(ts, write=False, normalize=True, z_sp=3,verbose=True,Npoints=20,usecol="_trace")
    try:
        tt = regions.showTime["%g"%ts]
    except:
        tt = t.copy()
    spikes1 = regions.calc_peaks(
        ts=ts,
        save=False,
        t=tt,
        zScores=zs,
        smooth=0
    )
    print(len(spikes1))
    if len(spikes1)==len(spikes):
        spikes = spikes1
        del spikes1
        break
    spikes = spikes1

In [ ]:
for ix in regions.df.index:
    y = regions.df.loc[ix, "_trace"]
    iis = spikes.query(f"roi=={ix}").index
    

In [ ]:

sl,fs,zs = regions.fast_filter_traces(ts, write=False, normalize=True, z_sp=3,verbose=True,Npoints=20,usecol="_trace")
try:
    tt = regions.showTime["%g"%ts]
except:
    tt = t.copy()
spikes = regions.calc_peaks(
    ts=ts,
    save=False,
    t=tt,
    zScores=zs,
    smooth=0
)
print(len(spikes))
    
    
for _ in range(4):
    print (_)
    if len(spikes)==0: break
    fitDF = []
    for i,ix in enumerate(regions.df.index[:]):
        iis = spikes.query(f"roi=={ix}").index
        if len(iis)==0: continue
        tmpdf = fit_spikes(
    #         t, regions.df.loc[ix,"_trace"],
           tt, sl[i]+fs[i],
           spikes.loc[iis],
           colorCode={"myShape":"C3"},
           half_width_threshold=ts*.12
          )
        tmpdf["roi"]= ix 
        fitDF += [tmpdf]
    fitDF = pd.concat(fitDF)
    if len(fitDF)==0: continue
    for _,row in fitDF.iterrows():
        ix = row.roi
        y  = regions.df.loc[ix,"_trace"]
        func = eval(row.fshape)
        yf,bkg = func(t, *row.pars, bkgsum=False)
        ff = np.isfinite(yf)
        y[ff] = bkg[ff]
    
    sl,fs,zs = regions.fast_filter_traces(ts, write=False, normalize=True, z_sp=3,verbose=True,Npoints=20,usecol="_trace")
    try:
        tt = regions.showTime["%g"%ts]
    except:
        tt = t.copy()
    spikes1 = regions.calc_peaks(
        ts=ts,
        save=False,
        t=tt,
        zScores=zs,
        smooth=0
    )
    print(len(spikes1))
    if len(spikes1)==len(spikes):
        spikes = spikes1
        del spikes1
        break
    spikes = spikes1

In [ ]:
fig, axs = plt.subplots(len(regions.df),1,figsize=(9,9), sharey=False, sharex=True)
sl1 = regions.fast_filter_traces(ts,
                                 write=False,
                                 normalize=True,
                                 z_sp=3,
                                 verbose=True,
                                 Npoints=20,
                                 usecol="_trace"
                                )[0]
for i,ix in enumerate(regions.df.index):
    ax = axs[i]
    ax.plot(t, regions.df.loc[ix, "trace"],lw=.4)
    ax.plot(t, regions.df.loc[ix,"_trace"],lw=.8)
    ax.plot(tt, sl1[i],lw=.8)
# plt.xlim(fitDF["loc"].min()-10, fitDF["loc"].max()+50)
plt.subplots_adjust(hspace=0.01)

In [ ]:
fitDF

In [ ]:
from islets.fitting import myShape

In [ ]:
tr = np.linspace(-2,10, 100)
# yr = multiShape(tr, pd.DataFrame([tmp]))+np.random.normal(size=len(tr))*.03

plt.figure()
r=1
for k in [1,3,.5]:
    yr = tr**k*np.exp(-r*tr)
    plt.plot(tr, yr/yr.max())
    
plt.plot(tr, myShape(tr,loc=0,decay=1, half_width=2),"--")
plt.plot(tr, myShape(tr,loc=0,decay=2, half_width=2),"--")
plt.ylim(1e-3,)
plt.yscale("log")

In [ ]:
# plt.figure()
# plt.plot(tr,yr)

fit_spikes(
     tr,yr,
     regions.calc_peaks(ts=5,t=tr,
                        zScores=yr.reshape(1,-1),
                        z_th=.1,
                        save=False),
                       colorCode={"myShape":"C3"},
                       plot=True,
#                        ax=axs[i],
#                        half_width_threshold=ts/12
                      )

In [ ]:
fitDF = pd.concat(fitDF, sort=False)

In [ ]:
%matplotlib notebook

In [ ]:
fig, axs = plt.subplots(len(regions.df),1,figsize=(10,10), sharey=False, sharex=True)
for i,ix in enumerate(regions.df.index):
    ax = axs[i]
    ax.plot(t, regions.df.trace[ix], lw=.4)
    ax.plot(t, regions.df.trace[ix]-multiShape(t,fitDF.query(f"roi=={ix}")), "k",lw=.3)
    ax.plot(tt,sl[i])
plt.tight_layout()
# plt.xlim(fitDF["loc"].min()-5, fitDF["loc"].max()+2)
plt.xlim(regions.time[0], regions.time[0]+100)
plt.subplots_adjust(hspace=0.01)

In [ ]:
err_scales = [(zs[i]/(fs[i]/sl[i]**.5)).mean() for i in range(len(zs))]

In [ ]:
plt.figure(figsize=(14,5))
ax = plt.subplot()
# plt.plot(t, y, lw=.6)
# plt.xlim(t[0], t[0]+100)



fit_spikes(t, y,
           spikes.query(f"roi=={itest}"),
           plot=True,
           # half_width_threshold = ts/5,
           # rel_amplitude_threshold = 0,
            ifit=3,
           colorCode={"myShape":"C1"},
           # colorCode={"myLogNorm":"C1"},
           ax=ax
          )
# plt.plot(tt,sl[i_test]/regions.)

In [ ]:
plt.hexbin(zs[i_test],(fs[i_test]/sl[i_test]**.5), bins="log", mincnt=1)
plt.grid()

In [ ]:
zs[i_test]

In [ ]:
ts = 1
sl,fs,zs = regions.fast_filter_traces(ts,
                                      write=False,
                                      normalize=False,
                                      z_sp=3
                                     )
t = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts]
except:
    tt = t.copy()
zth = 2
spikes = regions.calc_peaks(
    ts=ts,
    save=False,
    t=tt,
    zScores=zs,
#     smooth=0,
    verbose=True,
    z_th=zth
)

In [ ]:
# fig, axs = plt.subplots(len(regions.df),1,figsize=(9,9), sharey=False, sharex=True)
fig, ax = plt.subplots(1,1,figsize=(9,2.5), sharey=False, sharex=True)
for i,ix in enumerate(regions.df.index[:]):
#     ax = axs[i]
    t = regions.time.copy()
#     y = regions.df.loc[ix,"trace"]
    y = sl[i]+fs[i]
    ax.plot(t,y,"C0",lw=.8)
    ax.plot(tt,sl[i],"C1",lw=.8)
    df = spikes.query(f"roi=={ix}")
    for _,row in df.iterrows():
        ff = (t>(row["t0"]-row["peak half-width [s]"]*0.5)) & (t<(row["t0"]+row["peak half-width [s]"]*2))
        ax.plot(t[ff],y[ff],"C3")
        ax.plot(t[ff][0],y[ff][0]+y[ff].std()*5,"C3.")
#         plt.xlim(row["t0"]-row["peak half-width [s]"]*5-2, row["t0"]+row["peak half-width [s]"]*5+2)
#     if len(df)>30:
    wIron = int(np.round(ts/(t[1]-t[0])))
    if wIron%2==0: wIron+=1
    ys = lowPass(y,wIron,wIron,perc=20)
    ax.plot(t,ys,"C5",lw=.8)
    break

zs_ = (sl[i]+fs[i]-ys)/ys**.5
# plt.xlim(regions.time[0], regions.time[0]+300)

plt.subplots_adjust(hspace=0.01)

In [ ]:
spikes_ = regions.calc_peaks(
    ts=ts,
    save=False,
    t=tt,
    zScores=zs_.reshape(1,-1),
#     smooth=0,
    z_th=zth+1
)

df = spikes_.query(f"roi=={ix}")
for _,row in df.iterrows():
    row.t0-df
    ff = (t>(row["t0"]-row["peak half-width [s]"]*0.5)) & (t<(row["t0"]+row["peak half-width [s]"]*2))
    ax.plot(t[ff],y[ff],"C5",lw=.5)
    ax.plot(t[ff][0],y[ff][0]+y[ff].std()*5,
            "C5x",
#             "C5o",mfc="none"
           )


In [ ]:
frq,pw = power_spectrum(y,fr=regions.Freq)
plt.figure()
pw = np.abs(pw)
ff = frq>.4
plt.plot(frq[ff],pw[ff])

In [ ]:
from islets.numeric import lowPass

In [ ]:
lowPass?

In [ ]:
lowPass(y,19,)

In [ ]:
plt.figure()
plt.plot(t,regions.df.loc[ix,"trace"])
plt.plot(tt,sl[i])
plt.axvline(row.t0)
plt.plot(t,t>(row["t0"]-row["peak half-width [s]"]/.2))
plt.plot(t,t<(row["t0"]+row["peak half-width [s]"]*2))
plt.plot(t,)

plt.xlim(row["t0"]-row["peak half-width [s]"]*5-2, row["t0"]+row["peak half-width [s]"]*5+2)

In [ ]:
fig, axs = plt.subplots(3,1,figsize=(10,8), sharex=True, sharey=True)
fitDF = []
for ts,ax in zip([2,5],axs):
    spikes = regions.calc_peaks(ts,save=False)
    if len(fitDF):
        yy = y-multiShape(t,pd.concat(fitDF, ignore_index=True))
    else:
        yy = y
    fitDF += [fit_spikes(t, yy,
                       spikes.query(f"roi=={itest}"),
                       plot=True,
                       # half_width_threshold = ts/5,
                       # rel_amplitude_threshold = 0,
                       #  ifit=3,
                       colorCode={"myShape":"C1"},
                       # colorCode={"myLogNorm":"C1"},
                       ax=ax
                      )]
#     break

In [ ]:
ts = 5
spikes = regions.calc_peaks(ts,save=False)
fitDF += fit_spikes(t, y-multiShape(t,fitDF[-1]), spikes.query(f"roi=={itest}"),
                   plot=True,
                   half_width_threshold = ts/8,
#                    ifit=10,
                   colorCode={"myShape":"C1"}
#                    colorCode={"myLogNorm":"C1"}
                  )

In [ ]:
plt.figure()
plt.plot(t, multiShape(t,fitDF))

In [ ]:
fitDF["rel_ampl"] = fitDF.amplitude/fitDF.background

In [ ]:
fitDF

In [ ]:
mu

In [ ]:
for ts in [1,3,10,30,100,300]:
#     regions.calc_peaks(ts, z_th=4, verbose=True, smooth=None)
    regions.calc_peaks(ts)

In [ ]:
# ALLPEAKS = []
fig, axs = plt.subplots(2,3, figsize=(13,7), sharex=True, sharey=True)
for ts,ax in zip([1,3,10,30,100,300], axs.flat):
    df = regions.peaks["%g"%ts].copy()
    ax.hexbin(df["peak half-width [s]"], df["peak height [z-score]"], yscale="log", xscale="log", mincnt=1, gridsize=30, bins="log")
#     df["ts"] = ts
#     ALLPEAKS += [df]

In [ ]:
ts = 10
pdf = regions.peaks["%g"%ts]

In [ ]:
fitDF

In [ ]:
[fitDF.loc[i,"pars"].amplitude/fitDF.loc[i,"background"] for i in fitDF.index]

In [ ]:
# fitting of trace
ix = 64#regions.df.sort_values("zInterest", ascending=False).index[0]    
y  = regions.df.loc[ix,"trace"].copy()
t  = regions.time.copy()
ishow = 0
parDf = []
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    i0  = np.searchsorted(t , p.t0-p["peak half-width [s]"]*.5,)
    ie  = np.searchsorted(t , p.t0+p["peak half-width [s]"]*2+1)
    ycur = y[i0:ie]
    tcur = t[i0:ie]
    imax = np.argmax(ycur)
    res = pd.DataFrame(columns=["fshape","pars","score","background"])

    for fname in colorCode:
        fshape = eval(fname)
        pars0 = getInitialPars(tcur,ycur,p,fshape)
        try:
            try:
                opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,fshape),
                               bounds = [(pars0[j]*.05, pars0[j]*10) for j in range(len(pars0))]
                              #  method="Nelder-Mead"
                              )
            except:
                opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,fshape),
                               # bounds = [(pars0[j]*.05, pars0[j]*10) for j in range(len(pars0))],
                               # method="Nelder-Mead"
                              )
            yf,bkg = fshape(tcur,*opt.x, bkgsum=False, fillnan=True)
            pars = pars = pars0._replace(**dict(zip(pars0._asdict().keys(), opt.x)))
            res = res.append({"fshape": fshape.__name__,
                              "pars": pars,
                              "score":np.nanmean((ycur-yf-bkg)**2),
                              "background": np.nanmean(bkg)
                             }, ignore_index=True)
            
        except:
            continue
    res = res.sort_values("score").iloc[0]
    res.name = ip
    parDf += [res]
    ishow += 1
#     if ishow>=20: break

In [ ]:
%matplotlib notebook

In [ ]:
plt.figure(figsize=(9,5))
parDf = pd.DataFrame(parDf)
# ff = np.ones(len(parDf)).astype(bool)
plt.plot(t,y, "k",lw=.5,)
for c in ["darkred","C0"]:
    x = multiShape(t,parDf)
    plt.plot(t,x+2, lw=.5,c=c)
    plt.plot(t,y-x, lw=.5,c=c)
    w,h,t0,te = peak_widths(x, find_peaks(x)[0], rel_height=.9)
    t0,te = t[0]+t0/regions.Freq,t[0]+te/regions.Freq
    w = w/regions.Freq
    parDf = parDf[w<ts/2]
# plt.xlim(parDf[0].pars[3], parDf[-1].pars[3])
plt.xlim(t[np.where(x>0)[0][0]]-2, t[np.where(x>1e-3)[0][-1]]+20)

In [ ]:

x = multiShape(t,parDf)
w,h,t0,te = peak_widths(x, find_peaks(x)[0], rel_height=.9)
t0,te = t[0]+t0/regions.Freq,t[0]+te/regions.Freq
w = w/regions.Freq

In [ ]:
plt.figure()
plt.plot(w,[p.pars.amplitude/p.background for _,p in parDf.iterrows()],".")

In [ ]:
get_sep_th(w, plot=True, log=False)

In [ ]:
from scipy.signal import find_peaks, peak_widths

In [ ]:
plt.figure()
plt.plot(t,x)
w<
# plt.hlines(h,t0,te)
# plt.hlines(h,t0,te)
# plt.xlim(0,1000)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(sorted(w), np.arange(len(w)))
# plt.yscale("log")

In [ ]:
imax

In [ ]:
tr = np.linspace(0,10,1000)
plt.figure()
plt.plot(tr,myLogNorm(tr,amplitude=1,loc=1,s=1,scale=1,fillnan=False))
plt.plot(tr,myLogNorm(tr,amplitude=1,loc=1,s=1,scale=1,fillnan=True))
plt.plot(tr,myLogNorm(tr,amplitude=1,loc=1,s=1,scale=1,fillnan=False,offset=0,end=.1))
plt.plot(tr,myLogNorm(tr,amplitude=1,loc=1,s=1,scale=1,fillnan=True,offset=0,end=.1))
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.1,scale=1))
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=1,scale=2))
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=2))
plt.grid()

In [ ]:
ss = np.logspace(-1,.5,11)
for i,s in enumerate(ss):
    plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=s,scale=1, reshift=True), c=plt.cm.rainbow(i/len(ss)))
# plt.xlim(0,3)
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=1),"k--")
plt.yscale("log")
plt.ylim(1e-10,2)

In [ ]:
ss = np.logspace(-1,.5,11)
for i,s in enumerate(ss):
    plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=s), c=plt.cm.rainbow(i/len(ss)))
# plt.xlim(0,3)
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=1),"k--")
# plt.yscale("log")
# plt.ylim(1e-10,2)

In [ ]:
hw = .7
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,scale=2*hw))
plt.xlim(0,3)
plt.axhline(.5)
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=1),"k--")
# plt.yscale("log")
# plt.ylim(1e-10,2)

In [ ]:
LNParameters = namedtuple("Parameters", ['loc', 'scale', 's', 'amplitude', 'offset', 'end'])

In [ ]:
fitPars = pd.DataFrame(columns=['loc', 'scale', 's', 'amplitude', 'offset', 'end'])

In [ ]:
# fitting of trace with logNorm
fig, ax = plt.subplots(1,1,figsize=(9,4), sharex=True)
ix = regions.df.sort_values("zInterest", ascending=False).index[0]    
y  = regions.df.loc[ix,"trace"].copy()
ys = regions.df.loc[ix,"slower_%g"%ts].copy()
t  = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()

dtt = tt[1]-tt[0]
nrb = int(dtt/regions.Freq)
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
dt = dtt/max(1,nrb)
ax.plot(t, y, lw=.6)
ax.plot(tt, ys, lw=.8)
plt.tight_layout()

ishow = 0
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    is0 = np.searchsorted(tt, p.t0-p["peak half-width [s]"]*.5)
    i0  = np.searchsorted(t , p.t0-p["peak half-width [s]"]*.5,)
    ie  = np.searchsorted(t , p.t0+p["peak half-width [s]"]*2+1)
    ycur = y[i0:ie]
    tcur = t[i0:ie]
    imax = np.argmax(ycur)
#     ax.plot(tcur,ycur,"C2.", mfc="none")
    ax.plot(tcur[imax],ycur[imax],"ko")
#     r = guessDecayPars(ycur[imax-1:ie])[-1]
    pars0 = LNParameters(s=.2,
                         scale=p["peak half-width [s]"]*2,
                         amplitude=ycur[imax-1:imax+1].mean()-ys[is0],
                         loc=p.t0+p["peak half-width [s]"]*.2,
                         offset=ys[is0],
                         end=ys[is0]
                      )

    y0 = myLogNorm(tcur, *pars0)
    plt.plot(tcur, y0,"grey",lw=.7)
    opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,myLogNorm), bounds=[(pars0[j],pars0[j]) if j==2 else (None,None) for j in range(len(pars0))])
    opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,myLogNorm), bounds=[(pars0[j]*.5,pars0[j]*2) if j==2 else (None,None) for j in range(len(pars0))])
    pars = LNParameters(*opt.x)
#     try:
#         opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,myLogNorm))
#     except:
#         continue
    yf,bkg = myLogNorm(tcur,*opt.x, bkgsum=False)
    y[i0:ie][np.isfinite(yf)] -= yf[np.isfinite(yf)]
    plt.plot(tcur, yf+bkg, "k")
    plt.plot(tcur, y[i0:ie], "C1",lw=.5)
    fitPars.loc[ip] = opt.x
#     pdf.loc[[ip],"fitPars"] = [tuple(opt.x)]
    if ishow==0:
        tmin = t[i0]-10
    ishow += 1
#     if ishow>10: break
plt.xlim(tmin, tcur[-1]+10)

In [ ]:
fitPars

In [ ]:
LNParameters(*opt.x)

In [ ]:
pdf.loc[[ip],"fitPars"] = [range(len(pars0))]

In [ ]:
plt.plot(tr,myLogNorm(tr,*pars0._replace(loc=1), fillnan=False))

In [ ]:
p["ha"] = 1

In [ ]:
p

In [ ]:
pdf

In [ ]:
p.update(dict(zip(pars0._asdict().keys(), opt.x)))

In [ ]:
plt.plot(y0)

In [ ]:
# fitting of z
ix = regions.df.sort_values("zInterest", ascending=False).index[0]
if "zScore_%g"%ts not in regions.df:
    regions.fast_filter_traces(ts)
    
nrb = int(ts/regions.Freq)
fig, axs = plt.subplots(2,1,figsize=(9,6), sharex=True)

y = regions.df.loc[ix,"trace"].copy()
ys =regions.df.loc[ix,"slower_%g"%ts].copy()
z = regions.df.loc[ix,"zScore_%g"%ts].copy()
t = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
axs[0].plot(t, y, lw=.6)
axs[0].plot(tt, ys, lw=.8)
axs[1].plot(tt,z,"-", lw=.6)
dtt = tt[1]-tt[0]
plt.tight_layout()
# plt.xlim(t[0]-1,2000)

ishow = 0
for _,p in pdf.query(f"roi=={ix}").iterrows():
    i0 = np.searchsorted(tt,p.t0-1,)
    ie = np.searchsorted(tt,p.t0+p["peak half-width [s]"]*2+1)
    zcur = z[i0:ie]
    tcur = tt[i0:ie]
    imax = np.argmax(zcur)
    axs[1].plot(tcur,zcur,"C2.", mfc="none")
    axs[1].plot(tcur[imax],zcur[imax],"ko")
    pars0 = Parameters(decay=dtt/guessDecayPars(zcur[imax-1:ie])[-1],
                       half_width=p["peak half-width [s]"]*.8,
                       amplitude=z[i0:ie].max(),
                       loc=p.t0,
                       offset=0,
                       end=0
                      )

    z0 = myShape(tcur, *pars0)
    plt.plot(tcur, z0,"grey",lw=.7)
    try:
        opt = minimize(toMin, np.array(pars0), args=(tcur,zcur))
    except:
        continue
    zf = myShape(tcur,*opt.x)
    plt.plot(tcur, zf,"k")
    if ishow==0:
        tmin = tt[i0]-10
    ishow += 1
    if ishow>10: break
plt.xlim(tmin, tcur[-1]+10)

In [ ]:
from islets.numeric import guessDecayPars

In [ ]:
iHalf = i0+int(regions.Freq*(1+p["peak half-width [s]"]))

In [ ]:
pars = Parameters(*opt.x)

In [ ]:
%matplotlib inline

In [ ]:
plt.figure()
plt.plot(tt[i0:ie],z[i0:ie],"o", mfc="none")
plt.plot(tt[iHalf],z[iHalf],"x", mfc="none")

In [ ]:
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars),"k")
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars._replace(half_width=pars.half_width/2)))
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars._replace(decay=pars.decay*5)))
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars._replace(decay=pars.decay*5, half_width=pars.half_width*2.5)))
# plt.ioff()

In [ ]:
1/regions.Freq

### Other playing

In [ ]:
%matplotlib notebook

In [ ]:
ix = 57
coltype = "zScore"
plt.figure(figsize=(10,4))
it = 0
for ts in [1,3,5,10,30]:
    try: t = regions.showTime["%g"%ts]
    except: t = regions.time
    y = regions.df.loc[ix,"%s_%g"%(coltype,ts)]
    plt.plot(t, y+10*it,lw=.6)
    it += 1

In [ ]:
from islets.utils import autocorr

In [ ]:
plt.figure(figsize=(5,4))
for ts in [1,3,5,10,30]:
    try: t = regions.showTime["%g"%ts]
    except: t = regions.time
    dt = np.diff(t).mean()
    x = np.arange(int(30/dt))
    y = autocorr(regions.df.loc[ix,"%s_%g"%(coltype,ts)], x)
    c = plt.plot(x*dt,y,label=ts)[0].get_color()
    plt.plot([-1,x[np.argmin(y)]*dt],[y.min(), y.min()],c=c)
    plt.plot(x[np.argmin(y)]*dt,y.min(),"o",c=c, mfc="w")
plt.legend()

In [ ]:
ts = 3
regions.calc_peaks(ts)
pdf = regions.peaks["%g"%ts]

In [ ]:
# pdf.query(f"roi=={ix}")
ix = 57
nrb = 10
plt.figure(figsize=(10,4))
y = rebin(regions.df.loc[ix,"trace"],nrb).copy()
t = rebin(regions.time, nrb)
plt.plot(t, y, lw=.6)
ishow = 0
for _,p in pdf.query(f"roi=={ix}").iterrows():
    i0 = np.searchsorted(t,p.t0-1,)
    ie = np.searchsorted(t,p.t0+p["peak half-width [s]"]+4,)
#     break

    offset = y[i0]
#     scale = p["peak half-width [s]"]
    a = y[i0:ie].max()-offset
#     yf = myLogNorm(t,
#                       Ampl=a,
#                       loc=p.t0, 
#                       scale=scale,
#                        s=1,
#                       offset=offset)
    yf = myShape(t,loc=p.t0,decay=2.5,amplitude=a,half_width=p["peak half-width [s]"],offset=offset)
    plt.plot(t, yf,"k")
    plt.plot(t, y-yf+offset,"grey")
    ishow += 1
    if ishow>10: break
plt.xlim(t[0],t[ie]+30)

In [ ]:
%matplotlib inline

In [ ]:
def toMin(x):
    yf_ = myShape(t[i0:ie],*x,fillnan=False)
    return ((yf_-y[i0:ie])**2).sum()

In [ ]:
x0 = np.array([
    2.5,
    p["peak half-width [s]"],
    a,
    p["t0"],
    offset,
])
toMin(x0)

In [ ]:
from scipy.optimize import minimize

In [ ]:
opt = minimize(toMin,x0)

In [ ]:
tr = np.linspace(t[i0],t[ie],1000)

In [ ]:
plt.plot(t[i0:ie],y[i0:ie],"o-", mfc="w")
plt.axvline(p.t0,ls="--")
yshow = myShape(tr      ,loc=p.t0,decay=2.5,amplitude=a,half_width=p["peak half-width [s]"],offset=offset, fillnan=False)
yf0    = myShape(t[i0:ie],loc=p.t0,decay=2.5,amplitude=a,half_width=p["peak half-width [s]"],offset=offset, fillnan=False)
yf = myShape(tr,*opt.x, fillnan=False)
plt.plot(tr, yshow,"k")
# plt.plot(t[i0:ie], yf0,"ko", mfc="w")
plt.plot(tr,yf)
# plt.plot(t, y-yf+offset,"grey")
# plt.xlim(t[i0]-2,t[ie]+3)

In [ ]:
def toMin(x, t_, y_):
    yf_ = myShape(t_,*x,fillnan=False)
    return ((yf_-y_)**2).sum()

In [ ]:
%matplotlib notebook

In [ ]:
# pdf.query(f"roi=={ix}")
ix = 57
nrb = 10
plt.figure(figsize=(10,4))
y = rebin(regions.df.loc[ix,"trace"],nrb).copy()
t = rebin(regions.time, nrb)
plt.plot(t, y, lw=.6)
ishow = 0
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    
#     if ishow>=20: break
    i0 = np.searchsorted(t,p.t0-1,)
    ie = np.searchsorted(t,p.t0+p["peak half-width [s]"]+3,)
    x0 = np.array([
            2.5,                      # dacay
            p["peak half-width [s]"], # half width
            y[i0:ie].max()-y[i0],     # ampl
            p["t0"],                  # loc
            y[i0],                    # offset
            y[i0],                    # end
        ])
    tr = np.linspace(t[i0],t[ie],100)
    y0 = myShape(tr,*x0)
    plt.plot(tr, y0, "b", lw=.7)
    
    try:
        opt = minimize(toMin, x0, args=(t[i0:ie], y[i0:ie]))
        ishow += 1
    except:
         continue
    yf = myShape(tr,*opt.x)
    plt.plot(tr, yf,"C3")
    plt.plot(t[i0:ie], y[i0:ie]-myShape(t[i0:ie],*opt.x)+opt.x[-1],"lightgrey",lw=.5)
#     print (ishow, opt.x)
# plt.xlim(t[0],t[ie]+30)

In [ ]:
# pdf.query(f"roi=={ix}")
ix = 57
ts = .5
if "zScore_%g"%ts not in regions.df:
    regions.fast_filter_traces(ts)
    
nrb = 1
fig, axs = plt.subplots(2,1,figsize=(10,8))

In [ ]:
y = regions.df.loc[ix,"trace"].copy()
z = regions.df.loc[ix,"zScore_%g"%ts].copy()
t = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
axs[0].plot(t, y, lw=.6)
axs[1].plot(tt,z)

In [ ]:
from islets.numeric import power_spectrum

In [ ]:
fqs,pw = power_spectrum(y,regions.Freq/nrb)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(rebin(fqs[fqs>1.5],20), rebin(np.abs(pw)[fqs>1.5],20))

In [ ]:
ishow = 0
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    
#     if ishow>=20: break
    i0 = np.searchsorted(t,p.t0-1,)
    ie = np.searchsorted(t,p.t0+p["peak half-width [s]"]+3,)
    x0 = np.array([
            2.5,                      # dacay
            p["peak half-width [s]"], # half width
            y[i0:ie].max()-y[i0],     # ampl
            p["t0"],                  # loc
            y[i0],                    # offset
            y[i0],                    # end
        ])
    tr = np.linspace(t[i0],t[ie],100)
    y0 = myShape(tr,*x0)
    plt.plot(tr, y0, "b", lw=.7)
    
    try:
        opt = minimize(toMin, x0, args=(t[i0:ie], y[i0:ie]))
        ishow += 1
    except:
         continue
    yf = myShape(tr,*opt.x)
    plt.plot(tr, yf,"C3")
    plt.plot(t[i0:ie], y[i0:ie]-myShape(t[i0:ie],*opt.x)+opt.x[-1],"lightgrey",lw=.5)
#     print (ishow, opt.x)
# plt.xlim(t[0],t[ie]+30)

In [ ]:
# tr = np.linspace(t[i0],t[ie],100)
y0 = myShape(tr,*x0)

### Check for wiggles

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
th = get_sep_th(regions.df.peakValue.values, log=True, plot=True)

In [ ]:
# x = np.vstack([movie[:,point[0],point[1]] for point in regions.df.sort_values("peakValue",ascending=False).peak[:30]])
x = np.vstack([rebMovie[:,point[0],point[1]] for point in regions.df.sort_values("peakValue",ascending=False).peak[:30]])
# x = np.vstack(regions.df.trace)
# f,p = power_spectrum(x, fr=metadata.Frequency)
f,p = power_spectrum(x, fr=rebMovie.fr)
f,p = f[f>0.5],np.abs(p[f>0.5])

In [ ]:
n = 1
plt.plot(rebin(f,n),rebin(p,n))

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(x[0])

In [ ]:
i,j = regions.df.sort_values("peakValue",ascending=False).peak.iloc[0]

In [ ]:
plt.scatter(rebMovie[:,i,j],
            rebMovie[:,i+0,j-1],
            c=np.arange(len(rebMovie)),
            s=.4)
plt.gca().set_aspect("equal")

In [ ]:
# f,p = power_spectrum(np.vstack(data[:,0]),fr=metadata.Frequency,mean=False)

In [ ]:
# plt.boxplot(p[:,2:100],positions=f[2:100], showfliers=False);
# plt.plot(f[2:],p.mean(0)[2:],lw=.6)
# plt.xscale("symlog")

In [ ]:
# plt.plot(*sum([(f,pp) for pp in p[:10]],tuple()),lw=.3)

### Experiment with filtering the whole movie

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
th = get_sep_th(im.flatten(),log=True, plot=False)

In [ ]:
r = rs[0]

In [ ]:
r.update(movie)

In [ ]:
r.Freq

In [ ]:
r.fast_filter_traces(4,z_sp=0)

In [ ]:
n = 256
tmp = r.sosFilter.run(r.movie[:,:n,:n].reshape((r.movie.shape[0],-1)).T).T.reshape((-1,n,n))

In [ ]:
n1,n2 = int(.55*len(tmp)), int(.551*len(tmp))
print (r.time[n2]-r.time[n1])

In [ ]:
# show_movie(tmp,)

In [ ]:
np.percentile(tmp[n1:n2].flatten(),5)

In [ ]:
tmp[n1:n2].min()

In [ ]:
zMovie = (rebMovie[n1:n2]-tmp[n1:n2])/np.maximum(.04,tmp[n1:n2])**.5

In [ ]:
plt.hist(zMovie.flatten(),100,log=True);

In [ ]:
z = 1.5
im = (zMovie>z).sum(0)-(zMovie<-z).sum(0)
plt.imshow(im,cmap="bwr", vmin=-np.abs(im).max(), vmax=np.abs(im).max())

In [ ]:

# show_movie(
#     zMovie,
#     NTimeFrames=n2-n1,
#     log=False, 
#     autoadjust=False,
# #     out="save",
#     cmapArgs={"cmap":"bwr","vmin":-3,"vmax":3}
#           )

In [ ]:
regions = Regions(rebMovie, diag=True, gSig_filt=(3,5))

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(13,6))
for k,ax in zip(["mean","diff_std"],axs):
    ax.imshow(regions.statImages[k], norm=LogNorm())
    regions.plotEdges(ax=ax,image=False)
    regions.plotPeaks(ax=ax,image=False)

In [ ]:
regions.update(movie)

In [ ]:
regions.fast_filter_traces(10)

In [ ]:
n = 10
tmp = regions.sosFilter.run(regions.movie[:,:n,:n].reshape((regions.movie.shape[0],-1))).reshape((-1,n,n))

In [ ]:
from islets.utils import show_movie

In [ ]:
mshow = np.concatenate([
    rebin(movie[:,:n,:n],1200),
    rebin(tmp,1200),
    rebin(movie[:,:n,:n]-tmp,1200)
])

In [ ]:
mshow  = cmovie(mshow,fr=movie.fr)

In [ ]:
show_movie(mshow,
#           additionalPlot=lambda ax_: regions.plotEdges(ax=ax_, image=False),
#           log=False, 
          figScale=1,
          NTimeFrames=len(mshow),
#           out="save"
         )

In [ ]:
regions.calcTraces()

In [ ]:
regions.detrend_traces()

In [ ]:
regions.infer_gain(plot=True)

In [ ]:
regions.gain

In [ ]:
regions.gain = 1

In [ ]:
ts = 4
regions.fast_filter_traces(ts)

In [ ]:
x = np.vstack(regions.df["zScore_%g"%ts])
# x = rebin(x,100,1)
plt.imshow(x>3, aspect="auto")

In [ ]:
regions.df.columns

In [ ]:
regions.fast_filter_traces(10)